In [17]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [19]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [21]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [23]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [27]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_market_data[["price_change_percentage_24h", "price_change_percentage_7d"]])


In [29]:
# Create a DataFrame with the scaled data
scaled_df = pd.DataFrame(
    scaled_data,
    columns=["price_change_percentage_24h", "price_change_percentage_7d"],
    index=df_market_data.index
)

# Copy the crypto names from the original data
crypto_names = df_market_data.index.tolist()

# Set the coinid column as index (already done above, skip if needed)
# df_market_data = df_market_data.set_index("coin_id")  -- Not needed because it's already in index_col

# Display sample data
scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d
coin_id,,
bitcoin,0.508529,0.493193
ethereum,0.185446,0.934445
tether,0.021774,-0.706337
ripple,-0.040764,-0.810928
bitcoin-cash,1.193036,2.000959


---

### Find the Best Value for k Using the Original Data.

In [31]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))

In [35]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for k in k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, random_state=1)
    
    # 2. Fit the model to the data using `scaled_df`
    kmeans_model.fit(scaled_df)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans_model.inertia_)

C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Window

In [37]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia_values}


# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)


In [39]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve for KMeans Clustering")
elbow_plot


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 3

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [41]:
# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=4, random_state=1)


In [43]:
# Fit the K-Means model using the scaled data
kmeans.fit(scaled_df)


C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=1)

In [45]:
# Predict the clusters to group the cryptocurrencies using the scaled data
# Print the resulting array of cluster values.
cluster_labels = kmeans.predict(scaled_df)
print(cluster_labels)

[1 1 0 0 1 1 1 1 0 0 2 0 0 1 0 1 2 0 1 0 0 1 0 0 0 0 0 0 1 2 0 0 3 1 0 1 0
 0 1 0 0]


In [47]:
# Create a copy of the DataFrame
clustered_df = scaled_df.copy()


In [49]:
# Add a new column to the DataFrame with the predicted cluster
# Display sample data
clustered_df["cluster"] = cluster_labels
clustered_df.head()


,price_change_percentage_24h,price_change_percentage_7d,cluster
coin_id,,,
bitcoin,0.508529,0.493193,1
ethereum,0.185446,0.934445,1
tether,0.021774,-0.706337,0
ripple,-0.040764,-0.810928,0
bitcoin-cash,1.193036,2.000959,1


In [51]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
clustered_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols=["cluster"]
)


:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,cluster)

---

### Optimize Clusters with Principal Component Analysis.

In [59]:
# Create a PCA model instance and set `n_components=3`.#dif components from dataset
pca = PCA(n_components=2)

In [61]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.

market_data_pca = pca.fit_transform(scaled_df)
# View the first five rows of the DataFrame. 
market_data_pca[:5]

array([[ 0.70832473,  0.0108444 ],
       [ 0.79188247, -0.52962238],
       [-0.48405906,  0.51485209],
       [-0.60223751,  0.54458796],
       [ 2.25849563, -0.57128782]])

In [63]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.58482974, 0.41517026])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Total explained variance 1

In [65]:
# Create a new DataFrame with the PCA data.

pca_df = pd.DataFrame(
    market_data_pca,
    columns=["PC1", "PC2"]
)

# Copy the crypto names from the original data
crypto_names = df_market_data.index.tolist()

# Set the coinid column as index
pca_df["coin_id"] = crypto_names
pca_df = pca_df.set_index("coin_id")

# Display sample data
pca_df.head()

,PC1,PC2
coin_id,,
bitcoin,0.708325,0.010844
ethereum,0.791882,-0.529622
tether,-0.484059,0.514852
ripple,-0.602238,0.544588
bitcoin-cash,2.258496,-0.571288


---

### Find the Best Value for k Using the PCA Data

In [67]:
# Create a list with the number of k-values from 1 to 11
pca_k_values = list(range(1, 11))

In [69]:
# Create an empty list to store the inertia values
pca_inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in pca_k_values:
    pca_kmeans = KMeans(n_clusters=k, random_state=1)
    pca_kmeans.fit(pca_df)
    pca_inertia_values.append(pca_kmeans.inertia_)

C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Window

In [71]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {"k": pca_k_values, "inertia": pca_inertia_values}
# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(pca_elbow_data)

In [73]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_df.hvplot.line(x="k", y="inertia", title="PCA Elbow Curve for KMeans")

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**3


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it does not

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [75]:
# Initialize the K-Means model using the best value for k
pca_kmeans_model = KMeans(n_clusters=3, random_state=1)

In [77]:
# Fit the K-Means model using the PCA data
pca_kmeans_model.fit(pca_df)

C:\Users\greci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=3, random_state=1)

In [79]:
# Predict the clusters to group the cryptocurrencies using the PCA data
# Print the resulting array of cluster values.
pca_cluster_labels = pca_kmeans_model.predict(pca_df)
print(pca_cluster_labels)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 2 0 0 2 1 2 1 0
 0 1 0 0]


In [81]:
# Create a copy of the DataFrame with the PCA data
pca_clustered_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
# Display sample data
pca_clustered_df["cluster"] = pca_cluster_labels
pca_clustered_df.head()

,PC1,PC2,cluster
coin_id,,,
bitcoin,0.708325,0.010844,1
ethereum,0.791882,-0.529622,1
tether,-0.484059,0.514852,0
ripple,-0.602238,0.544588,0
bitcoin-cash,2.258496,-0.571288,1


In [83]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_clustered_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="cluster",
    hover_cols=["cluster"]
)

:NdOverlay   [cluster]
   :Scatter   [PC1]   (PC2,cluster)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [85]:
# Composite plot to contrast the Elbow curves

elbow_comparison = elbow_df.hvplot.line(x="k", y="inertia", title="Original Elbow Curve") + \
                   pca_elbow_df.hvplot.line(x="k", y="inertia", title="PCA Elbow Curve")

elbow_comparison

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [87]:
# Composite plot to contrast the clusters
original_clusters_plot = clustered_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols=["cluster"]
)

pca_clusters_plot = pca_clustered_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="cluster",
    hover_cols=["cluster"]
)

original_clusters_plot + pca_clusters_plot

:Layout
   .NdOverlay.I  :NdOverlay   [cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,cluster)
   .NdOverlay.II :NdOverlay   [cluster]
      :Scatter   [PC1]   (PC2,cluster)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** using fewer features resulted in similar clustering patterns while remaining visually distinct. Overall structure seemed similar showing PCA was effective at simplifying data without losing important information.